In [12]:
import metapy
import csv
import requests
import re
from bs4 import BeautifulSoup,Comment
from selenium import webdriver
import time
import json
import nltk
import pandas as pd
import gensim
import spacy
import math
import en_core_web_sm 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.model_selection import GridSearchCV
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
import numpy as np
import lda.datasets
import sklearn.feature_extraction.text as text
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics import pairwise_distances_argmin
from tkinter import *
import webbrowser
import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv('FinalDataset3.csv',encoding='cp1252').dropna()
df.head()

,Year,Term,Course_ID,Course_Name,Course_Description
0,2019,Fall,INLS 73,First-Year Seminar: Smart Cities,Topics and trends in sustainable and smart cit...
1,2019,Fall,INLS 89,First-Year Seminar: Special Topics,Special topics course; content will vary each ...
2,2019,Fall,INLS 151,Retrieving and Analyzing Information,Introduction to and application of the process...
3,2019,Fall,INLS 161,Tools for Information Literacy,Tools and concepts for information literacy. I...
4,2019,Fall,INLS 201,Foundations of Information Science,Examines the evolution of information science;...


In [14]:
data = df.Course_Description.values.tolist()

In [15]:
data = [re.sub("\'", "", sent) for sent in data]

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [17]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        #print(sent)
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
        #texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        #print(texts_out)
    return texts_out

In [18]:
data_words = list(sent_to_words(data))
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [19]:
#stop_no = open("numbers.txt")
#list_no=[]
#for line in stop_no : 
    #list_no.append(re.sub(r'[-,?!":;()|]\n',"",line,re.UNICODE).replace("'", "").strip())
STOPLIST = list(set(stopwords.words('english') +list(ENGLISH_STOP_WORDS)))
STOPLIST.extend(('.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','/','-'))

In [20]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        
                             stop_words=STOPLIST,             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',              
                            )

In [21]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [11]:
print(data_vectorized)

  (0, 136)	1
  (0, 90)	1
  (0, 419)	2
  (0, 181)	1
  (0, 344)	1
  (0, 259)	1
  (0, 88)	2
  (0, 188)	1
  (0, 348)	1
  (0, 40)	1
  (0, 413)	1
  (0, 410)	1
  (1, 359)	1
  (1, 426)	1
  (1, 65)	1
  (1, 69)	1
  (1, 378)	1
  (1, 410)	1
  (2, 145)	1
  (2, 127)	1
  (2, 420)	1
  (2, 311)	1
  (2, 14)	1
  (2, 201)	1
  (2, 188)	3
  :	:
  (1115, 391)	1
  (1115, 276)	1
  (1115, 268)	1
  (1115, 164)	1
  (1115, 142)	1
  (1115, 363)	1
  (1115, 203)	1
  (1115, 388)	1
  (1115, 389)	1
  (1115, 360)	1
  (1115, 69)	1
  (1116, 74)	1
  (1116, 225)	1
  (1116, 87)	1
  (1116, 405)	1
  (1116, 128)	1
  (1117, 94)	1
  (1117, 443)	1
  (1117, 326)	1
  (1117, 317)	1
  (1117, 258)	1
  (1117, 388)	1
  (1117, 186)	1
  (1117, 426)	1
  (1117, 410)	1


In [22]:
vectorizer.get_feature_names()

['access',
 'activity',
 'address',
 'advanced',
 'aesthetic',
 'africa',
 'african',
 'age',
 'algorithm',
 'america',
 'american',
 'analysis',
 'analyze',
 'ancient',
 'application',
 'apply',
 'approach',
 'architecture',
 'area',
 'art',
 'asia',
 'asian',
 'aspect',
 'assignment',
 'attention',
 'author',
 'available',
 'base',
 'basic',
 'body',
 'book',
 'british',
 'broad',
 'building',
 'business',
 'case',
 'central',
 'century',
 'challenge',
 'change',
 'city',
 'civil',
 'class',
 'classroom',
 'close',
 'cold',
 'collection',
 'colonial',
 'colonialism',
 'communication',
 'community',
 'comp',
 'comparative',
 'compose',
 'composition',
 'computational',
 'computer',
 'concept',
 'concern',
 'conduct',
 'conflict',
 'conquest',
 'consider',
 'construction',
 'contemporary',
 'content',
 'context',
 'control',
 'core',
 'course',
 'cover',
 'create',
 'credit',
 'critical',
 'criticism',
 'cross',
 'cultural',
 'culture',
 'current',
 'data',
 'database',
 'datum',
 'day

In [23]:
search_params = {'n_components': [10, 15,20], 'learning_decay': [.5, .7, .9]}
lda_model = LatentDirichletAllocation(random_state=100,n_topics=10)
model = GridSearchCV(lda_model, param_grid=search_params)
model.fit(data_vectorized)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=10, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [10, 15, 20], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
best_lda_model = model.best_estimator_
print("Chosen LDA Model is: ", model.best_params_)
lda_output = best_lda_model.transform(data_vectorized)
#print(lda_output)

Chosen LDA Model is:  {'learning_decay': 0.5, 'n_components': 10}


In [25]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8      -93.445999   26.558464       1        1  12.673554
6       18.180088   17.354450       2        1  12.556835
3       11.645267  124.343781       3        1  12.215216
9      116.837799 -163.339035       4        1  11.465036
2       97.045944  -52.738789       5        1  10.842895
7      131.798843   64.225082       6        1  10.381547
0       -3.155388  -86.059517       7        1   8.780604
4       -5.226307 -195.111267       8        1   8.277869
1     -111.671326  -97.971237       9        1   7.507191
5      204.113739  -49.756577      10        1   5.299251, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
26    Default  135.000000      available  135.000000  30.0000  30.0000
427   Default  126.000000        version  126.000000  29.0000  29.0000
174   Default  145.000000          honor  145.000000  28.0000  28.0000
388   Default  288.000000        student  288.000000  27.0000  27.0000
410   Default  197.000000          topic  197.000000  26.0000  26.0000
173   Default  213.000000        history  213.000000  25.0000  25.0000
188   Default  113.000000    information  113.000000  24.0000  24.0000
14    Default   69.000000    application   69.000000  23.0000  23.0000
343   Default  130.000000       research  130.000000  22.0000  22.0000
226   Default  117.000000     literature  117.000000  21.0000  21.0000
37    Default  108.000000        century  108.000000  20.0000  20.0000
311   Default   66.000000        process   66.000000  19.0000  19.0000
310   Default   78.000000        problem   78.000000  18.0000  18.0000
431   Default   90.000000            war   90.000000  17.0000  17.0000
10    Default  115.000000       american  115.000000  16.0000  16.0000
441   Default   66.000000          write   66.000000  15.0000  15.0000
282   Default   59.000000     permission   59.000000  14.0000  14.0000
11    Default   94.000000       analysis   94.000000  13.0000  13.0000
293   Default   74.000000      political   74.000000  12.0000  12.0000
389   Default  173.000000          study  173.000000  11.0000  11.0000
381   Default   68.000000          state   68.000000  10.0000  10.0000
190   Default   72.000000     instructor   72.000000   9.0000   9.0000
88    Default   88.000000         design   88.000000   8.0000   8.0000
56    Default   83.000000       computer   83.000000   7.0000   7.0000
3     Default   39.000000       advanced   39.000000   6.0000   6.0000
426   Default   34.000000           vary   34.000000   5.0000   5.0000
81    Default   67.000000          datum   67.000000   4.0000   4.0000
155   Default   50.000000         gender   50.000000   3.0000   3.0000
324   Default   32.000000          range   32.000000   2.0000   2.0000
184   Default  188.000000        include  188.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
352   Topic10    9.423494          scale   15.891055   2.4151  -4.3803
143   Topic10    7.964434           flow   13.888859   2.3815  -4.5485
24    Topic10   11.773038      attention   22.686455   2.2816  -4.1577
366   Topic10    9.227306          short   17.825537   2.2791  -4.4013
236   Topic10    5.522044       medicine   10.922901   2.2555  -4.9147
140   Topic10   16.148882        fiction   32.626571   2.2343  -3.8416
168   Topic10   18.146785         health   36.676518   2.2340  -3.7250
147   Topic10    7.629079         format   17.772353   2.0919  -4.5915
270   Topic10    8.369054           oral   19.761725   2.0784  -4.4989
157   Topic10   15.665202          genre   37.711876   2.0591  -3.8720
8     Topic10   10.406248      algorithm   26.718834   1.9946  -4.2811
378   Topic10   10.501203        special   32.334209   1.8130  -4.2720
55    Topic10    4.143090  computational   12.779268   1.8112  -5.

In [26]:
def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization
    tokens = list(sent_to_words(text))
    lemm = lemmatization(tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    vector = vectorizer.transform(lemm)
    topic_probability_scores = best_lda_model.transform(vector)
    topic = topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    #print(topic)
    #topic_keywords1 = pd.DataFrame(best_lda_model.components_)
    #topic_names1 = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]
    #topic_keywords1.columns = vectorizer.get_feature_names()
    #topic_keywords1.index = topic_names1

    #print(topic_keywords1)
    return topic, topic_probability_scores

In [27]:
topic_keywords = pd.DataFrame(best_lda_model.components_)
topic_names = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]
topic_keywords.columns = vectorizer.get_feature_names()
topic_keywords.index = topic_names

topic_keywords.to_csv(r'FeatureTopicDist.csv')
topic_keywords


,access,activity,address,advanced,aesthetic,africa,african,age,algorithm,america,...,western,wide,woman,women,work,world,write,writer,writing,year
Topic0,8.676861,10.612485,0.100035,0.100006,0.100008,0.100000,0.100004,1.118563,0.100007,0.100003,...,0.100003,0.100010,0.100001,0.100000,0.100003,3.661644,0.100005,0.189825,0.100006,0.100004
Topic1,0.100053,0.100016,0.100023,4.719429,0.100000,0.100001,0.100005,0.100015,10.303619,0.100001,...,0.100000,4.299540,0.100001,0.100004,1.515925,3.849368,0.100003,0.100002,0.100004,0.100002
Topic2,1.736289,0.155685,8.405301,0.100027,0.100000,0.100003,0.100000,0.100000,5.698283,0.100000,...,0.100000,0.100008,0.100000,0.100000,7.319114,6.548502,0.100002,0.100007,0.100000,0.100001
Topic3,0.100003,0.100010,1.257832,0.100004,8.177846,0.100004,0.100009,0.456659,0.100000,1.504206,...,0.100025,0.105984,0.100010,0.100007,35.564243,4.745608,0.100012,0.100020,6.453503,7.619990
Topic4,0.100000,0.100032,0.100001,2.901106,3.021997,0.100002,2.482046,0.100015,0.100005,0.100001,...,0.100002,0.100008,0.100006,1.677711,0.100008,0.100019,1.369417,0.100011,0.100020,5.439373
Topic5,1.886782,0.100031,0.100016,29.353979,0.100001,0.100002,0.100005,0.100024,10.298084,0.100004,...,3.209635,16.020736,0.100000,0.100000,8.951646,0.100003,12.326523,0.100011,0.100020,0.661389
Topic6,0.100007,7.531734,3.670610,1.425440,0.100000,0.100000,0.100000,0.100000,0.100003,0.100003,...,0.100000,0.100020,0.100011,0.639089,39.138006,0.100011,53.604021,2.124704,32.772391,5.529243
Topic7,0.100000,0.100004,0.100000,0.100000,0.100017,16.099967,2.695621,0.100057,0.100000,7.273526,...,8.990330,0.100005,0.100014,0.100007,8.170384,19.020791,0.100007,19.893704,0.100002,2.541642
Topic8,0.100004,0.100002,2.066161,0.100008,0.100121,0.100010,26.153185,0.100010,0.100000,24.522247,...,0.100000,3.973666,5.741337,9.334648,9.646848,29.603920,0.100009,2.191688,8.039838,5.808338
Topic9,0.100001,0.100001,0.100022,0.100001,0.100009,0.100011,1.069124,8.724656,0.100000,0.100009,...,0.100006,0.100022,22.458621,3.748534,3.493825,9.270133,0.100003,0.100028,1.134217,0.100017


In [30]:
def similar_documents(text, doc_topic_probs, documents = df, nlp=nlp, top_n=10):
    t1 = "".join(text)
    text = []
    text.append(t1)
    topic, x  = predict_topic(text)
    #print("x: {}".format(x))
    #print(len(topic))
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    print(euclidean_distances(x.reshape(1, -1), doc_topic_probs))
    print(x)
    #print(dists)
    #print(topic)
    doc_ids = np.argsort(dists)[:top_n]
    course_name = []
    course_id = []
    doc_probs = []
    distances = []
    dot_product = doc_ids.tolist()
    for i in dot_product : 
        course_id.append(df["Course_ID"][i])
        course_name.append(df["Course_Name"][i])
        
    print("topic distribution:")
    for i in dot_product:
        doc_probs.append(doc_topic_probs[i])
    
    print(doc_probs)
    
    print("checked_distsss:")
    distances = euclidean_distances(x.reshape(1, -1), doc_probs)
    for distanc in distances:
        for distance in distanc:
            print('{0:.10f}'.format(distance))
    
    return(list(zip(course_id, course_name)))
    
    #final_rank = list(zip(course_id, course_name))
    #searchf=Tk()

In [31]:
courses = similar_documents(text="Network protocols and protocol stacks. Included are discussions of protocol classes packet filtering address filtering network management and hardware such as protocol analyzers repeaters routers and bridges", doc_topic_probs=lda_output, documents = df, top_n=12)
print(courses)

[[0.73417315 1.05963235 0.70985019 ... 1.11354585 0.82738771 0.48693227]]
[[0.39817313 0.01250117 0.01250358 0.01250058 0.0125005  0.0125031
  0.50181502 0.01250037 0.01250108 0.01250147]]
topic distribution:
[array([0.39817313, 0.01250117, 0.01250358, 0.01250058, 0.0125005 ,
       0.0125031 , 0.50181502, 0.01250037, 0.01250108, 0.01250147]), array([0.44174151, 0.01111502, 0.01111564, 0.01111288, 0.01111122,
       0.01111134, 0.46935836, 0.01111111, 0.01111182, 0.01111111]), array([0.35537242, 0.00526517, 0.00526563, 0.00526576, 0.00526449,
       0.00526372, 0.60250913, 0.00526398, 0.00526507, 0.00526464]), array([0.31496276, 0.00666855, 0.13230741, 0.0066678 , 0.00666859,
       0.00666747, 0.5060554 , 0.00666691, 0.00666784, 0.00666726]), array([0.34290507, 0.00666712, 0.00666856, 0.00666941, 0.15336337,
       0.00666839, 0.45704991, 0.0066718 , 0.00666829, 0.0066681 ]), array([0.28551309, 0.00909389, 0.00909209, 0.00909204, 0.00909097,
       0.00909191, 0.64175241, 0.00909101, 

In [26]:
try:
    #top= Tk()
    top=Toplevel()
except:
    top=Toplevel()
    #top= Tk()
top.wm_title("Course Search")
photo = PhotoImage(file="images/img1.gif")
dimensions = "image size: %dx%d" % (photo.width(), photo.height())
w = Label(top,image=photo)
w.pack(side = "top")
photoimage = PhotoImage(file="images/UNCLogo.gif")
v = Label(top, image=photoimage)

v.pack(side = "bottom", fill = "both")
top.wm_iconbitmap("images/img1.ico")
f = Frame(top, width=600,height=350,bg="royal blue")
f.pack(fill=X, expand=True)
e1=Entry(top,bd=10,width=40,font=("Helvetica", 15))
e1.insert(END, 'search here')
e1.place(relx=0.5, rely=0.35, anchor=CENTER)
text = e1.get()

b1=Button(top,font=("Helvetica", 15),
          text="Click me!",command= lambda:similar_documents(text=list(e1.get()), doc_topic_probs=lda_output, documents = df, top_n=20))
b1.place(relx=0.5, rely=0.5, anchor=CENTER)
top.mainloop()

[[0.05       0.05       0.05       0.05       0.54999484 0.05000373
  0.05000142 0.05       0.05       0.05      ]]
118
